In [1]:
suppressPackageStartupMessages(require(dplyr))
suppressPackageStartupMessages(require(data.table))
suppressPackageStartupMessages(require(RColorBrewer))
suppressPackageStartupMessages(require(ggplot2))
#suppressPackageStartupMessages(require(colorRamp2))

**Parameters**

In [2]:
# Inputs
lmm <- "_lmm" #lmm <- "" #if "data/toplot_tfs.rds"
inputpath <- paste0("data/toplot_tfs",lmm,".rds")
color_palette_path <- "data/colors_palette.R"

# Outputs
outputpath <- "results/"

# Arguments
spectra_of_interest <- c("SPECTRA_110", "SPECTRA_13", "SPECTRA_130", "SPECTRA_24",
                         "SPECTRA_34", "SPECTRA_46", "SPECTRA_56", "SPECTRA_71",
                         "SPECTRA_86", "SPECTRA_98")

# Load data

In [3]:
# Load data
toplot <- readRDS(inputpath)
source(color_palette_path)

# Process to heatmap

**Data**

In [4]:
xx <-
    toplot %>%
    tidyr::pivot_wider(names_from = "annot", values_from = "z") %>% # values_from = "z" if object is toplot_tfs_lmm.rds
    tibble::column_to_rownames("target") %>%
    select(contains(spectra_of_interest)) %>%
    filter(!if_all(everything(), is.na))
head(xx)

,SPECTRA_110:STAT1:flu:T_CD8_NonNaive,SPECTRA_110:SP1:flu:T_CD8_NonNaive,SPECTRA_110:STAT1:SLE:T_CD8_NonNaive,SPECTRA_110:SP1:SLE:T_CD8_NonNaive,SPECTRA_110:STAT1:COVID:T_CD8_NonNaive,SPECTRA_110:STAT1:sepsis:T_CD8_NonNaive,SPECTRA_110:SP1:sepsis:T_CD8_NonNaive,SPECTRA_110:SP1:HNSCC:T_CD8_NonNaive,SPECTRA_110:SP1:COPD:T_CD8_NonNaive,SPECTRA_110:STAT1:COPD:T_CD8_NonNaive,⋯,SPECTRA_56:STAT1:BRCA:Plasma,SPECTRA_71:SP1:flu:T_CD4_Naive,SPECTRA_71:SP1:SLE:T_CD4_Naive,SPECTRA_71:SP1:COPD:T_CD4_Naive,SPECTRA_71:SP1:HNSCC:T_CD4_Naive,SPECTRA_71:SP1:UC:T_CD4_Naive,SPECTRA_98:STAT1:flu:T_CD8_Naive,SPECTRA_98:STAT1:SLE:T_CD8_Naive,SPECTRA_98:STAT1:sepsis:T_CD8_Naive,SPECTRA_98:STAT1:COVID:T_CD8_Naive
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OAS1,8.870,8.87,11.342,11.342,3.791,3.259,3.259,0.004,-0.853,-0.853,⋯,0.961,8.597,11.318,-1.731,0.883,0.268,8.369,10.775,2.399,3.776
IFIT3,4.980,NA,1.294,NA,2.276,0.645,NA,NA,NA,-0.046,⋯,0.695,NA,NA,NA,NA,NA,8.774,8.495,3.073,3.843
IFIT2,1.385,NA,-0.014,NA,1.164,0.163,NA,NA,NA,0.014,⋯,NA,NA,NA,NA,NA,NA,6.371,6.559,0.210,3.232
IFI6,9.309,NA,13.140,NA,5.071,3.046,NA,NA,NA,-0.475,⋯,0.940,NA,NA,NA,NA,NA,7.946,13.588,1.556,4.803
MX1,8.854,NA,11.655,NA,7.590,2.278,NA,NA,NA,-1.514,⋯,2.394,NA,NA,NA,NA,NA,8.052,11.531,2.057,NA
ADAR,0.978,NA,5.106,NA,-0.109,0.577,NA,NA,NA,-2.158,⋯,2.171,NA,NA,NA,NA,NA,1.527,5.931,-0.012,0.584


**Annotations**

In [5]:
col_annot <- data.frame(
    tf = as.factor(sapply(strsplit(names(xx), ":"), "[", 2)),
    disease = as.factor(sapply(strsplit(names(xx), ":"), "[", 3)),
#        SPECTRA = as.factor(sapply(strsplit(names(xx), ":"), "[", 1)),
    cell_type = as.factor(sapply(strsplit(names(xx), ":"), "[", 4)),
    row.names = names(xx)
) %>%
arrange(cell_type, tf, disease)

# Colors
my_colour_annot <- list(
    tf = tfs_colors[names(tfs_colors) %in% levels(col_annot$tf)],
    disease = disease_colors[names(disease_colors) %in% levels(col_annot$disease)],
    cell_type = cell_colors[names(cell_colors) %in% levels(col_annot$cell_type)]
)

my_breaks <- c(seq(-10, 0, length.out=ceiling(palette_length/2) + 1),
            seq(0.05, 15, length.out=floor(palette_length/2)))

# Plot heatmap

## Ordered by celltype

In [6]:
cell_type_order <- c("T_CD8_Naive","Plasma", "pDC", "DC", "Mono", "T_CD4_Naive", "T_CD8_NonNaive", "ILC")
tf_order <- c("STAT1", "SP1")
disease_order <- c(
  'RA', 'PS', 'PSA', 'CD', 'UC', 'SLE', 'MS',
  'asthma', 'COPD', 'cirrhosis', 'sepsis',
  'HIV', 'HBV', 'COVID', 'flu',
  'BRCA', 'NPC', 'HNSCC', 'CRC'
)

# Convert columns to factors with specified levels
df <- col_annot %>%
  mutate(
    cell_type = factor(cell_type, levels = cell_type_order),
    tf = factor(tf, levels = tf_order),
    disease = factor(disease, levels = disease_order)
  )

# Order the data frame by cell_type, tf, and disease
ordered_df <- df %>%
  arrange(cell_type, tf, disease)

# Print the ordered data frame
print(ordered_df)

figure_path = paste0(outputpath, "TF_genes_heatmap_order_celltype", lmm,".pdf")
pdf(figure_path, width = 15, height = 12)
a <- pheatmap::pheatmap(
    xx[, rownames(ordered_df)],
    border_color = "black",
    na_col = "black",
    color = my_color,
    breaks = my_breaks,
    annotation_colors = my_colour_annot,
    cluster_cols = FALSE,
    cluster_rows = FALSE,
    cellheight = 10,
    cellwidth = 10,
    annotation_col = col_annot,
    cex = 1,
    fontsize = 9,
    labels_col = col_annot$disease,
    filename = figure_path,
    main = "IFN_response",
    gaps_col = c(2, 6, 11, 17, 27, 29, 34, 38)
    )
print(a)
dev.off()

                                           tf   disease      cell_type
SPECTRA_98:STAT1:SLE:T_CD8_Naive        STAT1       SLE    T_CD8_Naive
SPECTRA_98:STAT1:sepsis:T_CD8_Naive     STAT1    sepsis    T_CD8_Naive
SPECTRA_98:STAT1:COVID:T_CD8_Naive      STAT1     COVID    T_CD8_Naive
SPECTRA_98:STAT1:flu:T_CD8_Naive        STAT1       flu    T_CD8_Naive
SPECTRA_56:STAT1:SLE:Plasma             STAT1       SLE         Plasma
SPECTRA_56:STAT1:flu:Plasma             STAT1       flu         Plasma
SPECTRA_56:STAT1:BRCA:Plasma            STAT1      BRCA         Plasma
SPECTRA_56:STAT1:HNSCC:Plasma           STAT1     HNSCC         Plasma
SPECTRA_56:STAT1:CRC:Plasma             STAT1       CRC         Plasma
SPECTRA_130:STAT1:PS:pDC                STAT1        PS            pDC
SPECTRA_130:STAT1:CD:pDC                STAT1        CD            pDC
SPECTRA_130:STAT1:SLE:pDC               STAT1       SLE            pDC
SPECTRA_130:STAT1:sepsis:pDC            STAT1    sepsis            pDC
SPECTR

pdf 
  4

## Separated cell types

In [7]:
cell_type_order <- c("T_CD8_Naive","Plasma", "pDC", "DC", "Mono", "T_CD4_Naive", "T_CD8_NonNaive", "ILC")
tf_order <- c("STAT1", "SP1")
disease_order <- c(
    'SLE',"flu",
  'RA', 'PS', 'PSA', 'CD', 'UC',  'MS',
  'asthma', 'COPD', 'cirrhosis', 'sepsis',
  'HIV', 'HBV', 'COVID',
  'BRCA', 'NPC', 'HNSCC', 'CRC'
)

# Convert columns to factors with specified levels
df <- col_annot %>%
  mutate(
    cell_type = factor(cell_type, levels = cell_type_order),
    tf = factor(tf, levels = tf_order),
    disease = factor(disease, levels = disease_order)
  )

# Order the data frame by cell_type, tf, and disease
ordered_df <- df %>%
  arrange(cell_type, tf, disease)

## Mono

In [8]:
celltype = "Mono"
metadata_df <- ordered_df %>%
  filter(cell_type == celltype) %>%
  arrange(cell_type, tf, disease)
heatmap_df <- xx[, rownames(metadata_df)] %>%
  filter(rowSums(is.na(.)) != ncol(.)) %>%
  mutate(na_order = case_when(
    is.na(.[, 1]) ~ 3,  # If NA in the first column, assign 3
    is.na(.[, ncol(.)]) ~ 2,  # If NA in the last column, assign 2
    TRUE ~ 1  # If no NA in the first or last column, assign 1
  )) %>%
  arrange(na_order, rowSums(is.na(.))) %>%  # Order by na_order and then by number of NAs
  select(-na_order)
figure_path = paste0(outputpath, "/TF_heatmap_per_celltype/TF_genes_heatmap_", celltype,"_", lmm,".pdf")
pdf(figure_path, width = 15, height = 12)
a <- pheatmap::pheatmap(
    heatmap_df,
    border_color = NA,
    na_col = "white",
    color = my_color,
    breaks = my_breaks,
    annotation_colors = my_colour_annot,
    cluster_cols = FALSE,
    cluster_rows = FALSE,
    cellheight = 10,
    cellwidth = 10,
    annotation_col = col_annot,
    cex = 1,
    fontsize = 6,
    labels_col = metadata_df$disease,
    filename = figure_path,
    main = celltype,
    gaps_col = c(9), 
    gaps_row = c(8, 21)
    )
print(a)
dev.off()

pdf 
  4

## T_CD8_NonNaive

In [9]:
celltype = "T_CD8_NonNaive"
metadata_df <- ordered_df %>%
  filter(cell_type == celltype) %>%
  arrange(cell_type, tf, disease)
heatmap_df <- xx[, rownames(metadata_df)] %>%
  filter(rowSums(is.na(.)) != ncol(.)) %>%
  mutate(na_order = case_when(
    is.na(.[, 1]) ~ 3,  # If NA in the first column, assign 3
    is.na(.[, ncol(.)]) ~ 2,  # If NA in the last column, assign 2
    TRUE ~ 1  # If no NA in the first or last column, assign 1
  )) %>%
  arrange(na_order, rowSums(is.na(.))) %>%  # Order by na_order and then by number of NAs
  select(-na_order)
figure_path = paste0(outputpath, "/TF_heatmap_per_celltype/TF_genes_heatmap_", celltype,"_", lmm,".pdf")
pdf(figure_path, width = 15, height = 12)
a <- pheatmap::pheatmap(
    heatmap_df,
    border_color = NA,
    na_col = "white",
    color = my_color,
    breaks = my_breaks,
    annotation_colors = my_colour_annot,
    cluster_cols = FALSE,
    cluster_rows = FALSE,
    cellheight = 10,
    cellwidth = 10,
    annotation_col = col_annot,
    cex = 1,
    fontsize = 6,
    labels_col = metadata_df$disease,
    filename = figure_path,
    main = celltype,
    gaps_col = c(7), 
    gaps_row = c(3,13)
    )
print(a)
dev.off()

pdf 
  4

## STAT1only celltypes

In [10]:
row_order <- c('IFI27', 'IFI6', 'MX1', 'ISG15',  'IFITM1', 'OAS1', "ISG20", "ADAR", "IFIT3","IFIT1",
'IFIT2','VIM',  'S100A10', 'PRF1',
                  'UPP1', 'TIMP1', 'LGALS3', 'LAP3', 'LY96', 'CD86', 'FGL2', 'FOS')

In [11]:
celltypes = c("T_CD8_Naive", "DC", "pDC", "Plasma")
metadata_df <- ordered_df %>%
  filter(cell_type %in% celltypes) %>%
  mutate(
    cell_type = factor(cell_type, levels = celltypes),
  )  %>%
  arrange(cell_type, tf, disease)
heatmap_df <- xx[row_order, rownames(metadata_df)] %>%
  filter(rowSums(is.na(.)) != ncol(.))
figure_path = paste0(outputpath, "/TF_heatmap_per_celltype/TF_genes_heatmap_STAT1only_celltypes_", lmm,".pdf")
pdf(figure_path, width = 15, height = 12)
a <- pheatmap::pheatmap(
    heatmap_df,
    border_color = NA,
    na_col = "white",
    color = my_color,
    breaks = my_breaks,
    annotation_colors = my_colour_annot,
    cluster_cols = FALSE,
    cluster_rows = FALSE,
    cellheight = 10,
    cellwidth = 10,
    annotation_col = col_annot,
    cex = 1,
    fontsize = 6,
    labels_col = metadata_df$disease,
    filename = figure_path,
    main = "STAT1_only celltypes",
    gaps_col = c(4,12,22), 
    gaps_row = c(10,11)
    )
print(a)
dev.off()

pdf 
  4

## SP1only celltypes

In [12]:
row_order <- c(
'GAPDH', 'ACTB', 'S100A6', 'IFITM3', 'ISG20','FTH1',
'OAS1',   'MX2','COX4I1', 'VIM', 'CALM1', 'SRSF7','FUS', 
                             
'LGALS1','MT2A','RAC2','ADGRG1',
               
'FBP1', 'S100A8', 'TYMP', 'TIMP1', 'LGALS3', 'ITGAX', 'SLC11A1', 'SPI1', 'PSAP', 'STXBP2', 'TKT', 'FGL2', 'LYZ', 'TNFSF13', 'TSPO', 'GSTP1', 'FOS',"SERPINA1" 


               
)

In [13]:
celltypes = c("T_CD4_Naive", "ILC")
metadata_df <- ordered_df %>%
  filter(cell_type %in% celltypes) %>%
  arrange(cell_type, tf, disease)
heatmap_df <- xx[row_order, rownames(metadata_df)] %>%
  filter(rowSums(is.na(.)) != ncol(.))
figure_path = paste0(outputpath, "/TF_heatmap_per_celltype/TF_genes_heatmap_SP1only_celltypes_", lmm,".pdf")
pdf(figure_path, width = 15, height = 12)
a <- pheatmap::pheatmap(
    heatmap_df,
    border_color = NA,
    na_col = "white",
    color = my_color,
    breaks = my_breaks,
    annotation_colors = my_colour_annot,
    cluster_cols = FALSE,
    cluster_rows = FALSE,
    cellheight = 10,
    cellwidth = 10,
    annotation_col = col_annot,
    cex = 1,
    fontsize = 6,
    labels_col = metadata_df$disease,
    filename = figure_path,
    main = "SP1_only celltypes",
    gaps_col = c(5), 
    gaps_row = c(6, 13)
    )
print(a)
dev.off()

pdf 
  4

## Ordered by TF

In [14]:
cell_type_order <- c("T_CD8_Naive","Plasma", "pDC", "DC", "Mono", "T_CD8_NonNaive", "T_CD4_Naive", "ILC")
tf_order <- c("STAT1", "SP1")
disease_order <- c(
    'SLE',"flu",
  'RA', 'PS', 'PSA', 'CD', 'UC',  'MS',
  'asthma', 'COPD', 'cirrhosis', 'sepsis',
  'HIV', 'HBV', 'COVID',
  'BRCA', 'NPC', 'HNSCC', 'CRC'
)

# Convert columns to factors with specified levels
df <- col_annot %>%
  mutate(
    cell_type = factor(cell_type, levels = cell_type_order),
    tf = factor(tf, levels = tf_order),
    disease = factor(disease, levels = disease_order)
  )

# Order the data frame by cell_type, tf, and disease
ordered_df <- df %>%
  arrange(tf, cell_type, disease)

figure_path = paste0(outputpath, "TF_genes_heatmap_order_TF", lmm,".pdf")
pdf(figure_path, width = 15, height = 12)
a <- pheatmap::pheatmap(
    xx[, rownames(ordered_df)],
    border_color = NA,
    na_col = "white",
    color = my_color,
    breaks = my_breaks,
    annotation_colors = my_colour_annot,
    cluster_cols = FALSE,
    cluster_rows = FALSE,
    cellheight = 10,
    cellwidth = 10,
    annotation_col = col_annot,
    cex = 1,
    fontsize = 9,
    labels_col = col_annot$disease,
    filename = figure_path,
    main = "IFN_response",
    gaps_col = c(2, 6, 11, 17, 22, 24, 29, 34, 38)
    )
print(a)
dev.off()

pdf 
  4

In [15]:
tf_list <- split(ordered_df, ordered_df$tf)

### STAT1

In [16]:
row_order <- c('IFI27', 'IFI6', 'MX1', 'ISG15',  'IFITM1', 'OAS1', "ISG20", "ADAR", "IFIT3","IFIT1",
'IFIT2','VIM',  'S100A10', 'PRF1',
                  'UPP1', 'TIMP1', 'LGALS3', 'LAP3', 'LY96', 'CD86', 'FGL2', 'FOS')

In [17]:
tf_name <- "STAT1"
tf_df <- tf_list[[tf_name]]

cell_type_order <- c("Mono", "T_CD8_Naive","T_CD8_NonNaive","pDC", "DC","Plasma")
tf_order <- c("STAT1", "SP1")
disease_order <- c(
    'SLE',"flu",
  'RA', 'PS', 'PSA', 'CD', 'UC',  'MS',
  'asthma', 'COPD', 'cirrhosis', 'sepsis',
  'HIV', 'HBV', 'COVID',
  'BRCA', 'NPC', 'HNSCC', 'CRC'
)

# Convert columns to factors with specified levels
tf_df <- tf_df %>%
  mutate(
    cell_type = factor(cell_type, levels = cell_type_order),
    tf = factor(tf, levels = tf_order),
    disease = factor(disease, levels = disease_order)
  )

# Order the data frame by cell_type, tf, and disease
tf_df <- tf_df %>%
  arrange(tf, cell_type, disease)

tf_xx <-   xx[row_order, rownames(tf_df)]
tf_xx <- tf_xx[rowSums(is.na(tf_xx)) != ncol(tf_xx), ]

figure_path <- paste0(outputpath, "TF_genes_heatmap_", tf_name, lmm,".pdf")
pdf(figure_path, width = 15, height = 12)
a <- pheatmap::pheatmap(
    tf_xx,
    border_color = NA,
    na_col = "white",
    color = my_color,
    breaks = my_breaks,
    annotation_colors = my_colour_annot,
    cluster_cols = FALSE,
    cluster_rows = FALSE,
    cellheight = 10,
    cellwidth = 10,
    annotation_col = tf_df,
    cex = 1,
    fontsize = 9,
    labels_col = tf_df$disease,
    filename = figure_path,
    main = "IFN_response",
    gaps_row = c(11,13,14),
    #gaps_col = c(5, 7, 9, 14, 20)
    gaps_col = c(9, 13, 20, 30, 38)
    )
print(a)
dev.off()

pdf 
  4

### SP1

In [18]:
row_order <- c('GAPDH', 'ACTB', 'S100A6', 'IFITM3', 'ISG20','FTH1',
'OAS1',   'MX2', 
               
'MT2A',
               
'S100A10',  
               
'COX4I1', 'VIM',
                             
'LGALS1',
               
'FBP1', 'S100A8', 'TYMP', 'TIMP1', 'LGALS3', 'ITGAX', 'SLC11A1', 'SPI1', 'PSAP', 'STXBP2', 'TKT', 'FGL2', 'LYZ', 'TNFSF13', 'TSPO', 'GSTP1', 'FOS',"SERPINA1", 

'CALM1', 'SRSF7','FUS', 
               
'RAC2',
               
'ADGRG1')

In [25]:
row_order <- c('GAPDH', 'ACTB', 'S100A6', 'IFITM3', 'ISG20','FTH1',
'OAS1',   'MX2', 'MT2A',
               

'RAC2',
'CALM1', 'SRSF7','FUS', 

               'ADGRG1', 
               
 'LGALS1',
               
'COX4I1', 'VIM',
   'S100A10',                           

               
'FBP1', 'S100A8', 'TYMP', 'TIMP1', 'LGALS3', 'ITGAX', 'SLC11A1', 'SPI1', 'PSAP', 'STXBP2', 'TKT', 'FGL2', 'LYZ', 'TNFSF13', 'TSPO', 'GSTP1',
               'FOS',"SERPINA1"


               

               

              )

In [26]:
tf_name <- "SP1"
tf_df <- tf_list[[tf_name]]

cell_type_order <- c("Mono", "T_CD8_NonNaive",  "T_CD4_Naive","ILC")
tf_order <- c("STAT1", "SP1")
disease_order <- c(
    'SLE',"flu",
  'RA', 'PS', 'PSA', 'CD', 'UC',  'MS',
  'asthma', 'COPD', 'cirrhosis', 'sepsis',
  'HIV', 'HBV', 'COVID',
  'BRCA', 'NPC', 'HNSCC', 'CRC'
)

# Convert columns to factors with specified levels
tf_df <- tf_df %>%
  mutate(
    cell_type = factor(cell_type, levels = cell_type_order),
    tf = factor(tf, levels = tf_order),
    disease = factor(disease, levels = disease_order)
  )

# Order the data frame by cell_type, tf, and disease
tf_df <- tf_df %>%
  arrange(tf, cell_type, disease)

tf_xx <-   xx[row_order, rownames(tf_df)]
tf_xx <- tf_xx[rowSums(is.na(tf_xx)) != ncol(tf_xx), ]

figure_path <- paste0(outputpath, "TF_genes_heatmap_", tf_name, lmm,".pdf")
pdf(figure_path, width = 15, height = 12)
a <- pheatmap::pheatmap(
    tf_xx,
    border_color = NA,
    na_col = "white",
    color = my_color,
    breaks = my_breaks,
    annotation_colors = my_colour_annot,
    cluster_cols = FALSE,
    cluster_rows = FALSE,
    cellheight = 10,
    cellwidth = 10,
    annotation_col = tf_df,
    cex = 1,
    fontsize = 9,
    labels_col = tf_df$disease,
    filename = figure_path,
    main = "IFN_response",
    #gaps_row = c(9,10,12,13,14,32,35),
    gaps_row = c(9,10,13,14, 18),
    #gaps_col = c(5,8, 10)
    gaps_col = c(10,15, 20)
    )
print(a)
dev.off()

pdf 
  4

In [80]:
tf_name <- "SP1"
tf_df <- tf_list[[tf_name]]

cell_type_order <- c("T_CD8_Naive","Plasma", "pDC", "DC", "Mono", "T_CD8_NonNaive", "T_CD4_Naive", "ILC")
tf_order <- c("STAT1", "SP1")
disease_order <- c(
    'SLE',"flu",
  'RA', 'PS', 'PSA', 'CD', 'UC',  'MS',
  'asthma', 'COPD', 'cirrhosis', 'sepsis',
  'HIV', 'HBV', 'COVID',
  'BRCA', 'NPC', 'HNSCC', 'CRC'
)

# Convert columns to factors with specified levels
tf_df <- tf_df %>%
  mutate(
    cell_type = factor(cell_type, levels = cell_type_order),
    tf = factor(tf, levels = tf_order),
    disease = factor(disease, levels = disease_order)
  )

# Order the data frame by cell_type, tf, and disease
tf_df <- tf_df %>%
  arrange(tf, cell_type, disease)

tf_xx <-   xx[row_order, rownames(tf_df)]
tf_xx <- tf_xx[rowSums(is.na(tf_xx)) != ncol(tf_xx), ]
tf_xx[is.na(tf_xx)] <- -15

figure_path <- paste0(outputpath, "TF_genes_heatmap_", tf_name, lmm,"2.pdf")
pdf(figure_path, width = 15, height = 12)
a <- pheatmap::pheatmap(
    tf_xx,
    border_color = "black",
    na_col = "black",
    color = my_color,
    breaks = my_breaks,
    annotation_colors = my_colour_annot,
    cluster_cols = FALSE,
    cluster_rows = TRUE,
    cellheight = 10,
    cellwidth = 10,
    annotation_col = tf_df,
    cex = 1,
    fontsize = 9,
    labels_col = tf_df$disease,
    filename = figure_path,
    main = "IFN_response",
    #gaps_row = c(6,8,9,15,32,35),
    gaps_col = c(5,8,10)
    )
print(a)
dev.off()

pdf 
  4